In [ ]:
!pip install germansentiment
!pip install spacy
!pip install -U sentence-transformers
!python -m spacy download de_core_news_sm

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np
from germansentiment import SentimentModel
import spacy
import torch
nlp = spacy.load("de_core_news_sm")

2023-08-25 07:26:50.766448: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-25 07:26:50.922375: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-25 07:26:51.492405: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-08-25 07:26:51.492507: W tensorflow/

In [25]:
ns = pd.read_csv("nonsmoking_label_preproc.csv")
ns.Sentence.to_csv("text/nonsmoking.txt", index=False, header=False)
insta = pd.read_csv("instagram_label_preproc.csv")
insta.Sentence.to_csv("text/insta.txt", index=False, header=False)
opti = pd.read_csv("optifast_label_preproc.csv")
opti.Sentence.to_csv("text/optifast.txt", index=False, header=False)
virtual = pd.read_csv("virtualcoach_label_preproc.csv")
virtual.Sentence.to_csv("text/health_coaching.txt", index=False, header=False)
woz = pd.read_csv("WoZ_label_preproc.csv")
woz.Sentence.to_csv("text/Darn_cat_woz.txt", index=False, header=False)
synth = pd.read_csv("gpt_output_for_pred.csv")
synth.Sentence.to_csv("text/synthetic.txt", index=False, header=False)
synth_labelled = pd.read_csv("synthetic_label_preproc.csv")
synth_labelled.Sentence.to_csv("text/synthetic_labelled.txt", index=False, header=False)

glohbcd = pd.read_csv("label_train_preprocessed.csv")
glo_test = pd.read_csv("test_label_preproc.csv")
glohbcd = pd.concat([glohbcd, glo_test])
glohbcd.Sentence.to_csv("text/glohbcd.txt", index=False, header=False)

In [6]:
len(glohbcd)

4724

In [7]:
model = SentimentModel()

In [14]:
ns_sents = model.predict_sentiment(ns.Sentence)
ns["sentiment"] = ns_sents
insta_sents = model.predict_sentiment(insta.Sentence)
insta["sentiment"] = insta_sents
opti_sents = model.predict_sentiment(opti.Sentence)
opti["sentiment"]= opti_sents
virtual_sents = model.predict_sentiment(virtual.Sentence)
virtual["sentiment"]= virtual_sents
woz_sents = model.predict_sentiment(woz.Sentence)
woz["sentiment"]= woz_sents
glohbcd_sents =  model.predict_sentiment(glohbcd.Sentence)
glohbcd["sentiment"]= glohbcd_sents
synth_labelled_sents =  model.predict_sentiment(synth_labelled.Sentence)
synth_labelled["sentiment"] = synth_labelled_sents

In [15]:
ns["sentiment"].value_counts(normalize=True)

negative    0.449624
neutral     0.403008
positive    0.147368
Name: sentiment, dtype: float64

In [16]:
insta["sentiment"].value_counts(normalize=True)

negative    0.431461
neutral     0.358427
positive    0.210112
Name: sentiment, dtype: float64

In [17]:
opti["sentiment"].value_counts(normalize=True)

neutral     0.493506
positive    0.298701
negative    0.207792
Name: sentiment, dtype: float64

In [18]:
virtual["sentiment"].value_counts(normalize=True)

neutral     0.417850
negative    0.318458
positive    0.263692
Name: sentiment, dtype: float64

In [19]:
woz["sentiment"].value_counts(normalize=True)

neutral     0.436620
negative    0.352113
positive    0.211268
Name: sentiment, dtype: float64

In [20]:
glohbcd["sentiment"].value_counts(normalize=True)

negative    0.461262
neutral     0.397968
positive    0.140771
Name: sentiment, dtype: float64

In [ ]:
synth_labelled["sentiment"].value_counts(normalize=True)

In [8]:
model = SentenceTransformer("Sahajtomar/German-semantic")

In [12]:
embeddings_list = []
list_names = ["GloHBCD", "Smoking Cessation", "Instagram", "Optifast", 
              "Health Coaching Dialogues", "DARN-CT WoZ", "Synthetic"]

embeddings_list.append(model.encode(list(glohbcd["Sentence"])))
embeddings_list.append(model.encode(list(ns["Sentence"])))
embeddings_list.append(model.encode(list(insta["Sentence"])))
embeddings_list.append(model.encode(list(opti["Sentence"])))
embeddings_list.append(model.encode(list(virtual["Sentence"])))
embeddings_list.append(model.encode(list(woz["Sentence"])))
embeddings_list.append(model.encode(list(synth_labelled["Sentence"])))

In [15]:
comp = {}

for index, entry in enumerate(embeddings_list):
    for jindex, jentry in enumerate(embeddings_list):
        cos_sim = util.cos_sim(entry, jentry)
        comp[list_names[index] + ":" + list_names[jindex]] = torch.mean(cos_sim).item()

print(comp)

{'GloHBCD:GloHBCD': 0.24437515437602997, 'GloHBCD:Smoking Cessation': 0.22323471307754517, 'GloHBCD:Instagram': 0.2311495691537857, 'GloHBCD:Optifast': 0.24484606087207794, 'GloHBCD:Health Coaching Dialogues': 0.2306177020072937, 'GloHBCD:DARN-CT WoZ': 0.2180624157190323, 'Smoking Cessation:GloHBCD': 0.22323469817638397, 'Smoking Cessation:Smoking Cessation': 0.28225696086883545, 'Smoking Cessation:Instagram': 0.21067538857460022, 'Smoking Cessation:Optifast': 0.2179681956768036, 'Smoking Cessation:Health Coaching Dialogues': 0.23284660279750824, 'Smoking Cessation:DARN-CT WoZ': 0.21271398663520813, 'Instagram:GloHBCD': 0.2311495840549469, 'Instagram:Smoking Cessation': 0.2106754183769226, 'Instagram:Instagram': 0.2711181938648224, 'Instagram:Optifast': 0.23482486605644226, 'Instagram:Health Coaching Dialogues': 0.22607660293579102, 'Instagram:DARN-CT WoZ': 0.21015195548534393, 'Optifast:GloHBCD': 0.24484606087207794, 'Optifast:Smoking Cessation': 0.2179681956768036, 'Optifast:Instagra

In [17]:
pd.DataFrame.from_dict(comp, orient="index")

,0
GloHBCD:GloHBCD,0.244375
GloHBCD:Smoking Cessation,0.223235
GloHBCD:Instagram,0.231150
GloHBCD:Optifast,0.244846
GloHBCD:Health Coaching Dialogues,0.230618
GloHBCD:DARN-CT WoZ,0.218062
Smoking Cessation:GloHBCD,0.223235
Smoking Cessation:Smoking Cessation,0.282257
Smoking Cessation:Instagram,0.210675
Smoking Cessation:Optifast,0.217968


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()

In [35]:
def calculate_ttr(tokens):
    vec = CountVectorizer()
    vectors = vec.fit_transform(tokens)
    types = len(vec.get_feature_names())
    ttr = types / len(tokens)
    return ttr

def get_lemma_info(ds, texts, lemmatize=False, batch_size=100):
    tokens = []
    sentence_lengths = []
    all_tokens = []

    for entry in texts:
        doc = nlp(entry)
        #tokens.append(' '.join([token.text for token in doc if not token.is_punct]))
        sentence_lengths.append(len(doc))
        if lemmatize:
            all_tokens.extend([token.lemma_ for token in doc if not token.is_punct and not token.is_space])
        else:
            all_tokens.extend([token.text for token in doc if not token.is_punct and not token.is_space])
        
    vec = CountVectorizer()
    vectors = vec.fit_transform(all_tokens)
    types = len(vec.get_feature_names())   
    
    counts = pd.DataFrame(vectors.toarray(),
                      columns=vec.get_feature_names())
    counted = counts.T.sort_values(by=0, ascending=False)
    counted["sum"] = counted.sum(axis=1)
    
    ttr_values = []
    batch_tokens = []
    
    print(all_tokens[0])
    print(len(all_tokens))
    
    for token in all_tokens:
        batch_tokens.append(token)
        if len(batch_tokens) >= batch_size:
            ttr = calculate_ttr(batch_tokens)
            ttr_values.append(ttr)
            batch_tokens = []
    
    ttr = types/len(all_tokens)
    if len(ttr_values) > 0:     
        mean_ttr = sum(ttr_values) / len(ttr_values)
        #print(ttr_values, len(ttr_values))
    else: 
        mean_ttr = ttr
    
    mean_lemma_occurrence = counted["sum"].mean()
    num_over_10_occurrences = len(counted[counted["sum"] >= 10])
    token_frame = pd.DataFrame(all_tokens)
    token_frame.to_csv(f"text/{ds}-lemmas.txt", header=None, index=None)
    print(f"""{ds} Total Tokens: {len(all_tokens)},
            Types: {types},
            TTR: {ttr},
            Mean Token Occurrence Count: {mean_lemma_occurrence}, 
            Number of Lemmas occurring more than 10 times: {num_over_10_occurrences},
            Mean Sentence Length (in tokens): {sum(sentence_lengths) / len(sentence_lengths)},
            Mean Standardized Type-Token Ratio (TTR): {mean_ttr}""")

In [ ]:
get_lemma_info("Smoking Cessation", ns["Sentence"])
get_lemma_info("Instagram", insta["Sentence"])
get_lemma_info("DARN-CT WoZ", woz["Sentence"])
get_lemma_info("GLoHBCD", glohbcd["Sentence"])
get_lemma_info("Optifast", opti["Sentence"])
get_lemma_info("Health Coaching Dialogues", virtual["Sentence"])
get_lemma_info("Synthetic", synth["Sentence"])

In [36]:
get_lemma_info("Smoking Cessation", ns["Sentence"], lemmatize=True)
get_lemma_info("Instagram", insta["Sentence"], lemmatize=True)
get_lemma_info("DARN-CT WoZ", woz["Sentence"], lemmatize=True)
get_lemma_info("GLoHBCD", glohbcd["Sentence"], lemmatize=True)
get_lemma_info("Optifast", opti["Sentence"], lemmatize=True)
get_lemma_info("Health Coaching Dialogues", virtual["Sentence"], lemmatize=True)
get_lemma_info("Synthetic", synth["Sentence"], lemmatize=True)

eigentlich
9257
Smoking Cessation Total Tokens: 9257,
            Types: 1732,
            TTR: 0.187101652803284,
            Mean Token Occurrence Count: 5.318706697459584, 
            Number of Lemmas occurring more than 10 times: 130,
            Mean Sentence Length (in tokens): 17.21954887218045,
            Mean Standardized Type-Token Ratio (TTR): 0.6908695652173912


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


es
15894


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Instagram Total Tokens: 15894,
            Types: 2708,
            TTR: 0.17037875928023152,
            Mean Token Occurrence Count: 5.807607090103398, 
            Number of Lemmas occurring more than 10 times: 205,
            Mean Sentence Length (in tokens): 21.42247191011236,
            Mean Standardized Type-Token Ratio (TTR): 0.7182278481012662
ich
616
DARN-CT WoZ Total Tokens: 616,
            Types: 274,
            TTR: 0.4448051948051948,
            Mean Token Occurrence Count: 2.2481751824817517, 
            Number of Lemmas occurring more than 10 times: 11,
            Mean Sentence Length (in tokens): 9.549295774647888,
            Mean Standardized Type-Token Ratio (TTR): 0.6683333333333334


...

Danke
6094
Health Coaching Dialogues Total Tokens: 6094,
            Types: 858,
            TTR: 0.1407942238267148,
            Mean Token Occurrence Count: 7.026806526806527, 
            Number of Lemmas occurring more than 10 times: 98,
            Mean Sentence Length (in tokens): 14.529411764705882,
            Mean Standardized Type-Token Ratio (TTR): 0.5978333333333335


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


ich
244224


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Synthetic Total Tokens: 244224,
            Types: 4263,
            TTR: 0.017455286949685535,
            Mean Token Occurrence Count: 57.25076237391508, 
            Number of Lemmas occurring more than 10 times: 910,
            Mean Sentence Length (in tokens): 11.9142416055577,
            Mean Standardized Type-Token Ratio (TTR): 0.5372440622440593


In [37]:
get_lemma_info("Synthetic_Test", synth_labelled["Sentence"], lemmatize=True)

ich
686
Synthetic Test Total Tokens: 686,
            Types: 148,
            TTR: 0.21574344023323616,
            Mean Token Occurrence Count: 4.635135135135135, 
            Number of Lemmas occurring more than 10 times: 17,
            Mean Sentence Length (in tokens): 11.256756756756756,
            Mean Standardized Type-Token Ratio (TTR): 0.5


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [26]:
3685/195552

0.018844092619865815

In [27]:
len(glohbcd)

4724

In [28]:
len(synth)

25910

In [38]:
get_lemma_info("GLoHBCD", glohbcd["Sentence"], lemmatize=True, batch_size=1000)
get_lemma_info("Synthetic", synth["Sentence"], lemmatize=True, batch_size=1000)

ich
73021


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


GLoHBCD Total Tokens: 73021,
            Types: 6496,
            TTR: 0.08896070993275906,
            Mean Token Occurrence Count: 11.175646551724139, 
            Number of Lemmas occurring more than 10 times: 689,
            Mean Sentence Length (in tokens): 18.809906858594413,
            Mean Standardized Type-Token Ratio (TTR): 0.4144931506849314
ich
244224


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Synthetic Total Tokens: 244224,
            Types: 4263,
            TTR: 0.017455286949685535,
            Mean Token Occurrence Count: 57.25076237391508, 
            Number of Lemmas occurring more than 10 times: 910,
            Mean Sentence Length (in tokens): 11.9142416055577,
            Mean Standardized Type-Token Ratio (TTR): 0.21875409836065576


In [ ]:
.